In [ ]:
# Header

In [ ]:
# environment/setup cell
import praw # python reddit api wrapper
import os
from random import randint # for random sampling
from statistics import stdev # sample standard deviation
from datetime import datetime
import plotly.offline as offline # for plotting
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [ ]:
# replace the <BLANK> with the relevant information
reddit = praw.Reddit(client_id='<BLANK>', 
                     client_secret='<BLANK>', 
                     user_agent='<BLANK>')

In [ ]:
# Method #1 - querying for new posts
queried_subreddits = set([])

all_of_reddit = reddit.subreddit('all')
#print(all_of_reddit.display_name)

for submission in reddit.subreddit('all').stream.submissions():
    try:
        subs = reddit.subreddit(submission.subreddit_name_prefixed.split('/')[-1]).subscribers # used to filter out submissions to subreddits that don't exist
        #print(submission.subreddit_name_prefixed.split('/')[-1])
        queried_subreddits.add((submission.subreddit_name_prefixed, subs))
        if len(queried_subreddits) >= 500:
            break
        else:
            #print(str(len(queried_subreddits)))
    except :
        pass

#print(len(queried_subreddits))

In [ ]:
# Method #2 - random sampling of 500 subreddits

path_to_subreddits_file = '\\\\?\\' + os.path.join(os.getcwd(), 'allsubreddits.txt' )

randomly_sampled_subreddits = set([])
random_integers = set([])

with open(path_to_subreddits_file, 'r') as infile:
    contents = infile.readlines()

    while len(randomly_sampled_subreddits) < 500:
        index = randint(0, len(contents))
        try:
            subreddit_name = '/'.join(contents[index].rstrip().split('/')[-3:-1])            
            print(subreddit_name)
            subs = reddit.subreddit(subreddit_name.split('/')[-1]).subscribers
            #print(subs)
            randomly_sampled_subreddits.add((subreddit_name,subs))
        except :
            pass
    
#print(randomly_sampled_subreddits)   

In [ ]:
queried_subreddit_subscribers = [ qs[1] for qs in queried_subreddits ]
sampled_subreddit_subscribers = [ rss[1] for rss in randomly_sampled_subreddits ]

queried_subscriber_mean = sum(queried_subreddit_subscribers)/len(queried_subreddit_subscribers)
queried_subscriber_sample_sd = stdev(queried_subreddit_subscribers)

sampled_subscriber_mean = sum(sampled_subreddit_subscribers)/len(sampled_subreddit_subscribers)
sampled_subscriber_sample_sd = stdev(sampled_subreddit_subscribers)

print('number of queried subreddits: {}'.format(str(len(queried_subreddit_subscribers))))
print('the queried subreddits had on average {} subscribers'.format(str(queried_subscriber_mean)))
print('the sample standard deviation of the number of subscribers for the queried subreddits was: {}'.format(str(queried_subscriber_sample_sd)))
print('the queried subreddits had on median of {} subscribers'.format(str(median(queried_subreddit_subscribers))))
print('the queried subreddits had a subscriber counts ranging from {} to {}'.format(str(min(queried_subreddit_subscribers)), str(max(queried_subreddit_subscribers))))
print('----')
print('number of randomly subsampled subreddits: {}'.format(str(len(sampled_subreddit_subscribers))))
print('the randomly subsampled subreddits had on average {} subscribers'.format(str(sampled_subscriber_mean)))
print('the sample standard deviation of the number of subscribers for the randomly subsampled subreddits was: {}'.format(str(sampled_subscriber_sample_sd)))
print('the randomly subsampled subreddits had on median of {} subscribers'.format(str(median(sampled_subreddit_subscribers))))
print('the randomly subsampled subreddits had a subscriber counts ranging from {} to {}'.format(str(min(sampled_subreddit_subscribers)), str(max(sampled_subreddit_subscribers))))

In [ ]:
#with open("list_of_queried_subreddits_and_their_subscriber_counts.txt",'w') as safeguard_file:
#    safeguard_file.write('#{}\n'.format(datetime.now().isoformat()))
#    for subreddit, subscribers in queried_subreddits:
#        safeguard_file.write('{}\t{}\n'.format(subreddit, str(subscribers)))

In [ ]:
trace0 = go.Box(
    y=queried_subreddit_subscribers,
    name="Queried Subreddits",
    boxmean=True
)
trace1 = go.Box(
    y=sampled_subreddit_subscribers,
    name="Randomly-Subsampled Subreddits",
    boxmean=True
)

data = [trace0, trace1]

layout = go.Layout(
    yaxis=dict(
        type='log',
        autorange=True
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)